**Mapas multicapa, clúster de marcas, ...**

In [ ]:
# Pinta un mapa de Galicia con varias capas
# Fondo tile map seleccionable: mínimo 3 capas
# Límites comarcais (mapas do servizo cartográfico (sergas))
# Marcas coas praias con bandeira azul en 2021 (portal "@bertos") + popup: nome da praia

# 1. Fai un primeiro mapa utilizando os datos desde o ficheiro CSV (para as praias) (sen clúster de marcas) 
# 2. Fai un segundo mapa utilizando os datos desde o ficheiro KML  (para as praias) (con clúster de marcas) 

In [ ]:
import geopandas as gpd
import pandas as pd
import folium

In [ ]:
# Ficheiros Shp da Xunta
#'https://www.sergas.es/Saude-publica/GIS-Comarcas'

filepath = '../../datasets/Comarcas/Comarcas.shp'
comarcas = gpd.read_file(filepath)

In [ ]:
# Ficheiro CSV con praias de bandeira azul
# https://abertos.xunta.gal/catalogo/cultura-ocio-deporte/-/dataset/0438/praias-galegas-con-bandeira-azul-2021
url = 'https://abertos.xunta.gal/catalogo/cultura-ocio-deporte/-/dataset/0438/praias-galegas-con-bandeira-azul-2021/001/descarga-directa-ficheiro.csv'
praiascsv = pd.read_csv(url,encoding='ISO-8859-15',sep=';')
praiascsv.head(3)

In [ ]:
# Ollo aos nomes das columnas <- teñen espazos
# Isto daríanos problemas á hora de referencialas
praiascsv.columns

In [ ]:
# Eliminamos espazos sobrantes ao inicio e final dos nomes
praiascsv.columns = praiascsv.columns.str.strip()
praiascsv.columns

In [ ]:
m = folium.Map(location=[43, -8.20],zoom_start=8, width=600, height=600)

# capa tiles
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('CartoDB Positron').add_to(m)

# capa comarcas
folium.GeoJson(comarcas, name='Comarcas').add_to(m)

# capa praias
for index, praia in praiascsv.iterrows():
    try:
        latitude = praia['COORDENADAS'].split(',')[0]
        lonxitude = praia['COORDENADAS'].split(',')[1]
        nome = praia['PRAIA']
        folium.Marker([latitude,lonxitude],popup=nome).add_to(m)
    except:
        pass 

folium.LayerControl().add_to(m)
m

In [ ]:
# Ficheiro KML con praias de bandeira azul
# https://abertos.xunta.gal/catalogo/cultura-ocio-deporte/-/dataset/0438/praias-galegas-con-bandeira-azul-2021
url = 'https://abertos.xunta.gal/catalogo/cultura-ocio-deporte/-/dataset/0438/praias-galegas-con-bandeira-azul-2021/002/descarga-directa-ficheiro.kml'

In [ ]:
import requests
import lxml # Pode ser necesario instalar o parser xml
from bs4 import BeautifulSoup

response = requests.get(url)
#print(response.text)
soup = BeautifulSoup(response.content,'lxml')

praiaskml = soup.find_all('placemark')

In [ ]:
import folium.plugins

m = folium.Map(location=[43, -8.20],zoom_start=8, width=600, height=600)

# capa tiles
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('CartoDB Positron').add_to(m)

# capa comarcas
folium.GeoJson(comarcas, name='Comarcas').add_to(m)

# clúster
marker_cluster = folium.plugins.MarkerCluster().add_to(m)


# capa praias
for praia in praiaskml:
    try:
        folium.Marker([praia.find('coordinates').text.split(',')[1], praia.find('coordinates').text.split(',')[0]],popup=praia.find('name').text).add_to(marker_cluster)      
    except:
        pass
m

folium.LayerControl().add_to(m)
m